In [1]:
import os
import openai
import pandas as pd
from Bio import Entrez, Medline
import time
import json
import ast
from tqdm import tqdm

In [2]:
alldata = []
with open('pubmed12345.txt',encoding='utf-8') as f:
    pmids = Medline.parse(f)
    for pmid in pmids:
        try:
            pid = pmid['PMID']
        except:
            pid = ''
        try:
            title = pmid['TI']
        except:
            title = ''
        try:
            abstract = pmid['AB']
        except:
            abstract = ''

        dic = {'PMID':pid,'TI':title,'AB':abstract}
        alldata.append(dic)

df = pd.DataFrame(alldata)
df.to_csv('output.csv',index=False)

In [3]:
import os
import openai
import pandas
from Bio import Entrez
import time

In [4]:
# 加载 .env 到环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [5]:
from pandas import read_csv

filename = "output.csv"

data = read_csv(filename)
print(data.shape)


(41, 3)


In [6]:
data['joined_content']="NA"
data['Note']="NA"

data['Chat_species']="NA"
data['token_species']="NA"
data['FoundYES_species']="NA"
data['FoundNO_species']="NA"
data['FoundNOTSURE_species']="NA"
data['FoundHOWEVER_species']="NA"

data['Chat_disease']="NA"
data['token_disease']="NA"
data['FoundYES_disease']="NA"
data['FoundNO_disease']="NA"
data['FoundNOTSURE_disease']="NA"
data['FoundHOWEVER_disease']="NA"

data['Chat_treatment']="NA"
data['token_treatment']="NA"
data['FoundYES_treatment']="NA"
data['FoundNO_treatment']="NA"
data['FoundNOTSURE_treatment']="NA"
data['FoundHOWEVER_treatment']="NA"

data['Chat_research_type']="NA"
data['token_research_type']="NA"
data['FoundYES_research_type']="NA"
data['FoundNO_research_type']="NA"
data['FoundNOTSURE_research_type']="NA"
data['FoundHOWEVER_research_type']="NA"

data['AllYES_summary']="NA"

In [7]:
data.columns.tolist


<bound method IndexOpsMixin.tolist of Index(['PMID', 'TI', 'AB', 'joined_content', 'Note', 'Chat_species',
       'token_species', 'FoundYES_species', 'FoundNO_species',
       'FoundNOTSURE_species', 'FoundHOWEVER_species', 'Chat_disease',
       'token_disease', 'FoundYES_disease', 'FoundNO_disease',
       'FoundNOTSURE_disease', 'FoundHOWEVER_disease', 'Chat_treatment',
       'token_treatment', 'FoundYES_treatment', 'FoundNO_treatment',
       'FoundNOTSURE_treatment', 'FoundHOWEVER_treatment',
       'Chat_research_type', 'token_research_type', 'FoundYES_research_type',
       'FoundNO_research_type', 'FoundNOTSURE_research_type',
       'FoundHOWEVER_research_type', 'AllYES_summary'],
      dtype='object')>

In [8]:
na_rows = data[data['AllYES_summary']=='NA'].index.tolist()

In [9]:
outputpath='outtest.csv'

In [10]:
from openai import OpenAI

In [18]:
MODEL="gpt-3.5-turbo-1106"
client = OpenAI(api_key="")

In [31]:
os.environ.get("OPENAI_API_KEY")

''

In [19]:
completion = client.chat.completions.create(
                model=MODEL, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant."},
                          {"role": "user", "content": "what is the capital of france"}]
            )

In [20]:
completion.choices[0].message.content

'The capital of France is Paris.'

In [14]:
    i=1
    temp_Title=data["TI"][i]
    temp_PMID=data["PMID"][i]
    #print(temp_PMID)
    #print(temp_Title)
    # Define the PMID of the article you want to retrieve
    pmid = temp_PMID

    # Use Entrez.efetch() to retrieve the article summary
    pmid_str = str(pmid)


        # Extract the article abstract from the record
        # try:
        #     handle = Entrez.efetch(db='pubmed', id=pmid_str, retmode='xml')
        #     record = Entrez.read(handle)
        #     abstract = record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
        # except KeyError:
        #     abstract = 'No abstract found'

    temp_Abstract=data["AB"][i]

        #print(temp_Abstract)
    joined_content=''.join(["Title: ",temp_Title,". Abstract: ",temp_Abstract])

    if len(joined_content) < 3000:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "qualified length"
    else:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "length is too long"
            joined_content = joined_content[0:3000]

    print(joined_content)

        #species#
    try:
            completion = client.chat.completions.create(
                model=MODEL, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that use human as primary research subject."},
                          {"role": "user", "content": joined_content}]
            )
    except KeyError:
            completion = 'completion ERROR'
    time.sleep(0.5)

    print(completion)
    if completion == 'completion ERROR':
            data["Chat_species"][i]='completion ERROR'
            data["token_species"][i]=0
    else:    
            data["Chat_species"][i]=completion.choices[0].message.content
            # data["token_species"][i]=completion.usage.total_tokens
 
    if "yes" in data["Chat_species"][i][0:3].lower():
            data["FoundYES_species"][i]="Found"
    else:
            data["FoundYES_species"][i]="NotFound"

    if "no." in data["Chat_species"][i][0:3].lower():
            data["FoundNO_species"][i]="Found"
    elif "no," in data["Chat_species"][i][0:3].lower():
            data["FoundNO_species"][i]="Found"
    else:
            data["FoundNO_species"][i]="NotFound"

    if "not sure" in data["Chat_species"][i][0:10].lower():
            data["FoundNOTSURE_species"][i]="Found"
    else:
            data["FoundNOTSURE_species"][i]="NotFound"

    if "however" in data["Chat_species"][i].lower():
            data["FoundHOWEVER_species"][i]="Found"
    else:
            data["FoundHOWEVER_species"][i]="NotFound"

    data.to_csv(outputpath,sep=',',index=True,header=True)

/tmp/ipykernel_3312/2204197878.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["joined_content"][i] = joined_content
/tmp/ipykernel_3312/2204197878.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Note"][i] = "length is too long"


Title: [Surgical treatment of primary liver cancer:a report of 10 966 cases].. Abstract: Objective: To summarize the experience of surgical treatment of primary liver cancer. Methods: The clinical data of 10 966 surgically managed cases with primary liver cancer, from January 1986 to December 2019 at Hepatobiliary Center, the First Affiliated Hospital of Nanjing Medical University, were retrospectively analyzed. The life table method was used to calculate the survival rate and postoperative recurrence rate. Log-rank test was used to compare the survival process of different groups, and the Cox regression model was used for multivariate analysis. In addition, 2 884 cases of hepatocellular carcinoma(HCC) with more detailed follow-up data from 2009 to 2019 were selected for survival analysis. Among 2 549 patients treated with hepatectomy, there were 2 107 males and 442 females, with an age of (56.6±11.1) years (range: 20 to 86 years). Among 335 patients treated with liver transplantation,

/tmp/ipykernel_3312/2204197878.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Chat_species"][i]=completion.choices[0].message.content
/tmp/ipykernel_3312/2204197878.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["FoundYES_species"][i]="Found"
/tmp/ipykernel_3312/2204197878.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["FoundNO_species"][i]="NotFound"
/tmp/ipykernel_3312/2204197878.py:70: SettingWithCo

In [40]:
data.head(1)

,PMID,TI,AB,joined_content,Note,Chat_species,token_species,FoundYES_species,FoundNO_species,FoundNOTSURE_species,...,FoundNO_treatment,FoundNOTSURE_treatment,FoundHOWEVER_treatment,Chat_research_type,token_research_type,FoundYES_research_type,FoundNO_research_type,FoundNOTSURE_research_type,FoundHOWEVER_research_type,AllYES_summary
0,34853654,Anatomical vs nonanatomical liver resection fo...,BACKGROUND: The long-term survival of patients...,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [41]:
for i in na_rows:
    try:
        print(i)
        temp_Title=data["TI"][i]
        temp_PMID=data["PMID"][i]
        #print(temp_PMID)
        #print(temp_Title)
        # Define the PMID of the article you want to retrieve
        pmid = temp_PMID

        # Use Entrez.efetch() to retrieve the article summary
        pmid_str = str(pmid)


        # Extract the article abstract from the record
        # try:
        #     handle = Entrez.efetch(db='pubmed', id=pmid_str, retmode='xml')
        #     record = Entrez.read(handle)
        #     abstract = record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
        # except KeyError:
        #     abstract = 'No abstract found'

        temp_Abstract=data["AB"][i]

        #print(temp_Abstract)
        joined_content=''.join(["Title: ",temp_Title,". Abstract: ",temp_Abstract])

        if len(joined_content) < 3000:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "qualified length"
        else:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "length is too long"
            joined_content = joined_content[0:3000]

        print(joined_content)

        #species#
        try:
            completion = openai.chat.completions.create(
                model=MODEL, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that use human as primary research subject."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_species"][i]='completion ERROR'
            data["token_species"][i]=0
        else:    
            data["Chat_species"][i]=completion.choices[0].message.content
            data["token_species"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_species"][i][0:3].lower():
            data["FoundYES_species"][i]="Found"
        else:
            data["FoundYES_species"][i]="NotFound"

        if "no." in data["Chat_species"][i][0:3].lower():
            data["FoundNO_species"][i]="Found"
        elif "no," in data["Chat_species"][i][0:3].lower():
            data["FoundNO_species"][i]="Found"
        else:
            data["FoundNO_species"][i]="NotFound"

        if "not sure" in data["Chat_species"][i][0:10].lower():
            data["FoundNOTSURE_species"][i]="Found"
        else:
            data["FoundNOTSURE_species"][i]="NotFound"

        if "however" in data["Chat_species"][i].lower():
            data["FoundHOWEVER_species"][i]="Found"
        else:
            data["FoundHOWEVER_species"][i]="NotFound"

        data.to_csv(outputpath,sep=',',index=True,header=True)
        
        #disease#
        try:
            completion = client.chat.completions.create(
                model=MODEL, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that involve patients with hepatocellular carcinoma."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_disease"][i]='completion ERROR'
            data["token_disease"][i]=0
        else:    
            data["Chat_disease"][i]=completion.choices[0].message.content
            data["token_disease"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_disease"][i][0:3].lower():
            data["FoundYES_disease"][i]="Found"
        else:
            data["FoundYES_disease"][i]="NotFound"

        if "no." in data["Chat_disease"][i][0:3].lower():
            data["FoundNO_disease"][i]="Found"
        elif "no," in data["Chat_disease"][i][0:3].lower():
            data["FoundNO_disease"][i]="Found"
        else:
            data["FoundNO_disease"][i]="NotFound"

        if "not sure" in data["Chat_disease"][i][0:10].lower():
            data["FoundNOTSURE_disease"][i]="Found"
        else:
            data["FoundNOTSURE_disease"][i]="NotFound"

        if "however" in data["Chat_disease"][i].lower():
            data["FoundHOWEVER_disease"][i]="Found"
        else:
            data["FoundHOWEVER_disease"][i]="NotFound"

        data.to_csv(outputpath,sep=',',index=True,header=True)
        
        #treatment#
        try:
            completion = client.chat.completions.create(
                model=MODEL, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: The title and abstract must explicitly mention that the study involves treatment of patients with one or more of the following topic: anatomical vs nonanatomical liver resection."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_treatment"][i]='completion ERROR'
            data["token_treatment"][i]=0
        else:    
            data["Chat_treatment"][i]=completion.choices[0].message.content
            data["token_treatment"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_treatment"][i][0:3].lower():
            data["FoundYES_treatment"][i]="Found"
        else:
            data["FoundYES_treatment"][i]="NotFound"

        if "no." in data["Chat_treatment"][i][0:3].lower():
            data["FoundNO_treatment"][i]="Found"
        elif "no," in data["Chat_treatment"][i][0:3].lower():
            data["FoundNO_treatment"][i]="Found"
        else:
            data["FoundNO_treatment"][i]="NotFound"

        if "not sure" in data["Chat_treatment"][i][0:10].lower():
            data["FoundNOTSURE_treatment"][i]="Found"
        else:
            data["FoundNOTSURE_treatment"][i]="NotFound"

        if "however" in data["Chat_treatment"][i].lower():
            data["FoundHOWEVER_treatment"][i]="Found"
        else:
            data["FoundHOWEVER_treatment"][i]="NotFound"

        data.to_csv(outputpath,sep=',',index=True,header=True)
        
        #research_type#
        try:
            completion = client.chat.completions.create(
                model=model_select, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets our criteria or not. I want you to only reply with yes, no, or not sure, and followed with reasons. The criteria is: studies that was original research instead of reviews, protocol, or case report with less than five patients."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        time.sleep(0.5)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_research_type"][i]='completion ERROR'
            data["token_research_type"][i]=0
        else:    
            data["Chat_research_type"][i]=completion.choices[0].message.content
            data["token_research_type"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_research_type"][i][0:3].lower():
            data["FoundYES_research_type"][i]="Found"
        else:
            data["FoundYES_research_type"][i]="NotFound"

        if "no." in data["Chat_research_type"][i][0:3].lower():
            data["FoundNO_research_type"][i]="Found"
        elif "no," in data["Chat_research_type"][i][0:3].lower():
            data["FoundNO_research_type"][i]="Found"
        else:
            data["FoundNO_research_type"][i]="NotFound"

        if "not sure" in data["Chat_research_type"][i][0:10].lower():
            data["FoundNOTSURE_research_type"][i]="Found"
        else:
            data["FoundNOTSURE_research_type"][i]="NotFound"

        if "however" in data["Chat_research_type"][i].lower():
            data["FoundHOWEVER_research_type"][i]="Found"
        else:
            data["FoundHOWEVER_research_type"][i]="NotFound"
        
        if data["FoundYES_species"][i]=="Found" and data["FoundYES_disease"][i]=="Found" and data["FoundYES_treatment"][i]=="Found" and data["FoundYES_research_type"][i]=="Found":
            data["AllYES_summary"][i]="Yes"
        else: 
            data["AllYES_summary"][i]="No"
            
        data.to_csv(outputpath,sep=',',index=True,header=True)
    except:
        pass

0
Title: Anatomical vs nonanatomical liver resection for solitary hepatocellular carcinoma: A systematic review and meta-analysis.. Abstract: BACKGROUND: The long-term survival of patients with solitary hepatocellular carcinoma (HCC) following anatomical resection (AR) vs non-anatomical resection (NAR) is still controversial. It is necessary to investigate which approach is better for patients with solitary HCC. AIM: To compare perioperative and long-term survival outcomes of AR and NAR for solitary HCC. METHODS: We performed a comprehensive literature search of PubMed, Medline (Ovid), Embase (Ovid), and Cochrane Library. Participants of any age and sex, who underwent liver resection, were considered following the following criteria: (1) Studies reporting AR vs NAR liver resection; (2) Studies focused on primary HCC with a solitary tumor; (3) Studies reporting the long-term survival outcomes (> 5 years); and (4) Studies including patients without history of preoperative treatment. The 

/tmp/ipykernel_2359/668022897.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["joined_content"][i] = joined_content
/tmp/ipykernel_2359/668022897.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Note"][i] = "qualified length"


1
Title: [Surgical treatment of primary liver cancer:a report of 10 966 cases].. Abstract: Objective: To summarize the experience of surgical treatment of primary liver cancer. Methods: The clinical data of 10 966 surgically managed cases with primary liver cancer, from January 1986 to December 2019 at Hepatobiliary Center, the First Affiliated Hospital of Nanjing Medical University, were retrospectively analyzed. The life table method was used to calculate the survival rate and postoperative recurrence rate. Log-rank test was used to compare the survival process of different groups, and the Cox regression model was used for multivariate analysis. In addition, 2 884 cases of hepatocellular carcinoma(HCC) with more detailed follow-up data from 2009 to 2019 were selected for survival analysis. Among 2 549 patients treated with hepatectomy, there were 2 107 males and 442 females, with an age of (56.6±11.1) years (range: 20 to 86 years). Among 335 patients treated with liver transplantatio

/tmp/ipykernel_2359/668022897.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["joined_content"][i] = joined_content
/tmp/ipykernel_2359/668022897.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Note"][i] = "length is too long"


2
Title: Anatomic resection improved the long-term outcome of hepatocellular carcinoma patients with microvascular invasion: A prospective cohort study.. Abstract: BACKGROUND: The long-term effect of anatomic resection (AR) is better than that of non-anatomic resection (NAR). At present, there is no study on microvascular invasion (MVI) and liver resection types. AIM: To explore whether AR improves long-term survival in patients with hepatocellular carcinoma (HCC) by removing the peritumoral MVI. METHODS: A total of 217 patients diagnosed with HCC were enrolled in the study. The surgical margin was routinely measured. According to the stratification of different tumor diameters, patients were divided into the following groups: ≤ 2 cm group, 2-5 cm group, and > 5 cm group. RESULTS: In the 2-5 cm diameter group, the overall survival (OS) of MVI positive patients was significantly better than that of MVI negative patients (P = 0.031). For the MVI positive patients, there was a statistical

...

/tmp/ipykernel_2359/668022897.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["joined_content"][i] = joined_content
/tmp/ipykernel_2359/668022897.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Note"][i] = "qualified length"


39
Title: Hepatic lobectomies in children: experience of a center in the light of changing management of malignant liver tumors.. Abstract: Hepatic resection is the main treatment modality for hepatic tumors in childhood. Advances in diagnostic technique, preoperative preparation, surgical technique, and postoperative management increased the success rate. The aim of this study is to report our experience in hepatic lobectomy, which is relatively rare procedure in childhood. Medical records of 25 patients who underwent hepatic lobectomy between January 1977 and June 2002 were reviewed retrospectively. Age, gender, diagnosis, physical examination findings, results of preoperative laboratory investigations, radiological examination, resectability criteria, preoperative biopsies, chemotherapies, radiotherapies, postoperative pathological results, incisions, operation technique, intraoperative transfusions, drains used, antibiotic prophylaxes, and intraoperative and postoperative complicat

/tmp/ipykernel_2359/668022897.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["joined_content"][i] = joined_content
/tmp/ipykernel_2359/668022897.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Note"][i] = "qualified length"


40
Title: [Laparoscopic treatment of solid and cystic tumors of the liver. Study of 33 cases].. Abstract: AIM OF THE STUDY: Laparoscopic liver surgery is still in its early stages. The aim of this study was to report our experience in the laparoscopic management of solid and cystic liver tumours. PATIENTS AND METHODS: From April 1991 to December 1999, 32 patients with various lesions of the liver underwent laparoscopic liver surgery. One group of patients presented with cysts (n = 15) (11 giant solitary cysts and 4 polycystic liver diseases) and one group of patients presented with solid tumours (n = 18): focal nodular hyperplasia (n = 8), haemangioma (n = 6), adenoma (n = 2), isolated metastasis from a colonic cancer (n = 1) and hepatocellular carcinoma (n = 1). Fifteen cyst fenestrations and eighteen liver resections were performed via a laparoscopic approach including 1 right lobectomy, 5 left lateral segmentectomies, 2 subsegmentectomies IVb, 1 segmentectomy III and 9 non-anatomica

/tmp/ipykernel_2359/668022897.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["joined_content"][i] = joined_content
/tmp/ipykernel_2359/668022897.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Note"][i] = "qualified length"


In [42]:
data.head(2)

,PMID,TI,AB,joined_content,Note,Chat_species,token_species,FoundYES_species,FoundNO_species,FoundNOTSURE_species,...,FoundNO_treatment,FoundNOTSURE_treatment,FoundHOWEVER_treatment,Chat_research_type,token_research_type,FoundYES_research_type,FoundNO_research_type,FoundNOTSURE_research_type,FoundHOWEVER_research_type,AllYES_summary
0,34853654,Anatomical vs nonanatomical liver resection fo...,BACKGROUND: The long-term survival of patients...,Title: Anatomical vs nonanatomical liver resec...,qualified length,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,33412628,[Surgical treatment of primary liver cancer:a ...,Objective: To summarize the experience of surg...,Title: [Surgical treatment of primary liver ca...,length is too long,Yes. This publication meets the criteria as it...,NA,Found,NotFound,NotFound,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
